# Master Thesis


In [1]:
from utils import *
from network import *
import pandas as pd
import json

In [2]:
net = Network()
#utils = Utils("./config.ini")

## Bipartite album-artists network 

In [2]:

df,albums_json=utils.load_data_top_5000_albums()
bipartite_G,nb_album = net.build_bipartite_network(df,albums_json)

# jazz Album projection network 

In [17]:
albums = None
with open('./cleaning/album-final.json',encoding="UTF-8") as f:
        albums = json.load(f)      

discogs_graph = net.build_album_projection_network(albums)
net.get_network_info(discogs_graph)

In [19]:
net.save_graph(discogs_graph,"../networks/gephi/discogs_album_projection.gexf")

nb nodes:  3467
nb edges:  245757
Min node degree:  0
Max node degree:  747
Avg node degree:  141.7692529564465


In [34]:
zero_degree_node = [node for node in discogs_graph.nodes if discogs_graph.degree(node) == 0]
len(zero_degree_node)

# Jazz collaborators network 

In [10]:
albums = None
with open('./cleaning/album-final.json',encoding="UTF-8") as f:
        albums = json.load(f)   
        
collaborators_projection_network = net.build_collaborators_projection_network(albums)
net.get_network_info(collaborators_projection_network)           

nb nodes:  17180
nb edges:  31369
Min node degree:  1
Max node degree:  233
Avg node degree:  3.651804423748545


In [18]:
net.save_graph(collaborators_projection_network,"../networks/gephi/discogs_collaborators_projection.gexf")

In [15]:
top50musicians = sorted(collaborators_projection_network.degree, key=lambda x: x[1], reverse=True)[0]

In [16]:
top50musicians

(252966, 233)

# Montreux

In [31]:
mjf = None
with open('../data/mjf/mfj_export.json',encoding="UTF-8") as f:
        mjf = json.load(f)      
results = mjf['concerts']        

In [33]:
def build_network(results) :  
    G = nx.Graph()
    for result in results:
      event = int("999"+str(result['id']) )
      participants = result['musicians']
      if not G.has_node(event) :
           G.add_node(event, name=result['name'],type='event')

      for musician in participants:
          m_name = musician['name']
          m_id = musician['id']
          if not G.has_node(m_id) :
                G.add_node(m_id, name=m_name,type='musician')

          if not G.has_edge(event,m_id) :
                G.add_edge(event,m_id,weight = 1)    
          else :
                G[event][m_id]['weight'] = G[event][m_id]['weight'] + 1  
    return G   

In [34]:
mjf_bipartite_graph = build_network(results)

In [35]:
net.get_network_info(mjf_bipartite_graph)

nb nodes:  22493
nb edges:  26552
Min node degree:  0
Max node degree:  83
Avg node degree:  2.360912283821633


In [36]:
net.save_graph(mjf_bipartite_graph,"../networks/gephi/mjf_bipartite_graph.gexf")